<a href="https://colab.research.google.com/github/TC2008B-Team5/Multiagent-Systems-T5/blob/main/MESA_Team5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install -U 'mesa'

In [11]:
!pip install seaborn; solara; matplotlib; ipywidgets

                                                                                
 Usage: solara [OPTIONS] COMMAND [ARGS]...                                      
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --help      Show this message and exit.                                      │
╰──────────────────────────────────────────────────────────────────────────────╯
╭─ Commands ───────────────────────────────────────────────────────────────────╮
│ create            Quickly create a solara script or project.                 │
│ deploy                                                                       │
│ run               Run a Solara app.                                          │
│ search            Build search index based on ssg output                     │
│ ssg               Static site generation                                     │
│ staticbuild       Experime

In [12]:
# Import Mesa modules
from mesa import Agent, Model
from mesa.agent import AgentSet
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

# Import visualization modules
from mesa.visualization import SolaraViz, make_space_component
import solara
import numpy as np

In [13]:
from mesa.agent import Agent
from mesa.model import Model

class CarAgent(Agent):
    """An agent representing a car moving from a parking lot to a destination parking lot."""

    def __init__(self, model: Model, start_pos: tuple[int, int], destination_pos: tuple[int, int]):
        super().__init__(model)
        self.pos = start_pos
        self.destination_pos = destination_pos
        self.active = True  # Indicates if the car is still moving
        self.path = []
        self.calculate_path()

    def calculate_path(self):
        """Calculate a path from start_pos to destination_pos avoiding buildings and following road directions."""
        # Implement a pathfinding algorithm that respects buildings and road directions
        self.path = self.find_path(self.pos, self.destination_pos)

    def find_path(self, start: tuple[int, int], goal: tuple[int, int]) -> list[tuple[int, int]]:
        """Find a valid path using A* algorithm."""
        open_set = {start}
        came_from = {}
        g_score = {start: 0}
        f_score = {start: self.heuristic(start, goal)}

        while open_set:
            current = min(open_set, key=lambda pos: f_score.get(pos, float('inf')))
            if current == goal:
                return self.reconstruct_path(came_from, current)

            open_set.remove(current)
            for neighbor in self.get_valid_neighbors(current):
                tentative_g_score = g_score[current] + 1  # Assumes uniform cost
                if tentative_g_score < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, goal)
                    open_set.add(neighbor)
        return []

    def heuristic(self, a: tuple[int, int], b: tuple[int, int]) -> int:
        """Heuristic function for A* (Manhattan distance)."""
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def reconstruct_path(self, came_from: dict, current: tuple[int, int]) -> list[tuple[int, int]]:
        """Reconstruct the path from start to goal."""
        total_path = [current]
        while current in came_from:
            current = came_from[current]
            total_path.append(current)
        total_path.reverse()
        return total_path

    def get_valid_neighbors(self, pos: tuple[int, int]) -> list[tuple[int, int]]:
        """Get neighbors that are valid for movement."""
        neighbors = self.model.grid.get_neighborhood(pos, moore=False, include_center=False)
        valid_neighbors = []
        for neighbor in neighbors:
            if self.is_valid_move(pos, neighbor):
                valid_neighbors.append(neighbor)
        return valid_neighbors

    def is_valid_move(self, from_pos: tuple[int, int], to_pos: tuple[int, int]) -> bool:
        """Check if the move from from_pos to to_pos is valid."""
        if self.model.grid.out_of_bounds(to_pos):
            return False
        cell_contents = self.model.grid[to_pos]
        if cell_contents is not None:
            return False  # Cell is occupied by another agent
        # Check for buildings and road direction
        if self.model.is_building(to_pos):
            return False  # Can't move into a building
        if not self.model.is_valid_road_direction(from_pos, to_pos):
            return False  # Road direction doesn't allow this move
        return True

    def step(self):
        """Advance the agent by one step."""
        if self.active and self.path:
            next_pos = self.path.pop(0)
            self.model.grid.move_agent(self, next_pos)
            if next_pos == self.destination_pos:
                self.active = False
                self.remove()
        else:
            self.active = False
            self.remove()

    def remove(self):
        """Remove the car agent from the model and grid."""
        if self.pos is not None:
            self.model.grid.remove_agent(self)
        super().remove()

In [14]:
class TrafficLightAgent(Agent):
    """An agent representing a traffic light."""

    def __init__(self, model: Model, pos: tuple[int, int]):
        super().__init__(model)
        self.pos = pos
        self.state = 'Green'  # Initial state
        self.timer = 0
        self.durations = {'Green': 5, 'Yellow': 2, 'Red': 5}
        self.model.grid.place_agent(self, pos)

    def step(self):
        """Advance the traffic light by one step."""
        self.timer += 1
        if self.timer >= self.durations[self.state]:
            self.change_state()
            self.timer = 0

    def change_state(self):
        """Cycle through traffic light states."""
        if self.state == 'Green':
            self.state = 'Yellow'
        elif self.state == 'Yellow':
            self.state = 'Red'
        elif self.state == 'Red':
            self.state = 'Green'

In [15]:
class CityModel(Model):
    """A model representing a city grid with cars, buildings, parking lots, and traffic lights."""

    def __init__(self, num_cars=5, width=24, height=24, seed=None):
        super().__init__(seed=seed)

        self.num_cars = num_cars
        self.grid = MultiGrid(width, height, torus=False)

        # Initialize property layers
        self.buildings_layer = np.full((width, height), False, dtype=bool)
        self.parking_lot_layer = np.full((width, height), False, dtype=bool)
        self.road_direction_layer = np.full((width, height), None, dtype=object)

        # Set up buildings, parking lots, and roads
        self.setup_environment()

        # Agent sets
        self.car_agents = AgentSet([], random=self.random)
        self.traffic_light_agents = AgentSet([], random=self.random)

        # Create traffic lights (if any)
        self.create_traffic_lights()

        # Create cars
        self.create_cars()

    def setup_environment(self):
        """Set up buildings, parking lots, and road directions."""
        # Place buildings
        self.setup_buildings()
        # Place parking lots
        self.setup_parking_lots()
        # Set road directions
        self.setup_road_directions()

    def setup_buildings(self):
        """Place buildings on the grid."""
        # Example: Buildings occupy multiple cells
        buildings_positions = [
            # Building 1
            [(5, 5), (5, 6), (6, 5), (6, 6)],
            # Building 2
            [(10, 10), (10, 11), (11, 10), (11, 11)],
            # Add more buildings as needed
        ]
        for building in buildings_positions:
            for pos in building:
                x, y = pos
                self.buildings_layer[x, y] = True  # Mark cell as building

    def setup_parking_lots(self):
        """Place parking lots on the grid."""
        parking_lot_positions = [
            (0, 0),
            (0, self.grid.height - 1),
            (self.grid.width - 1, 0),
            (self.grid.width - 1, self.grid.height - 1),
            # Add more parking lots as needed
        ]
        for pos in parking_lot_positions:
            x, y = pos
            self.parking_lot_layer[x, y] = True  # Mark cell as parking lot

    def setup_road_directions(self):
        """Set road directions on the grid."""
        # Example: Define road directions as 'N', 'S', 'E', 'W'
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                # Skip buildings and parking lots
                if self.buildings_layer[x, y] or self.parking_lot_layer[x, y]:
                    continue
                # Set road direction (example logic)
                if (x + y) % 2 == 0:
                    self.road_direction_layer[x, y] = 'E'  # East-bound road
                else:
                    self.road_direction_layer[x, y] = 'N'  # North-bound road

    def create_cars(self):
        """Create car agents with start and destination positions from parking lots."""
        # Get positions of parking lots
        parking_lot_positions = [
            (x, y) for x in range(self.grid.width) for y in range(self.grid.height)
            if self.parking_lot_layer[x, y]
        ]
        for _ in range(self.num_cars):
            if len(parking_lot_positions) < 2:
                raise ValueError("Not enough parking lots to assign start and destination.")
            start_pos = self.random.choice(parking_lot_positions)
            destination_pos = self.random.choice(parking_lot_positions)
            while destination_pos == start_pos:
                destination_pos = self.random.choice(parking_lot_positions)
            # Create CarAgent with start and destination positions
            car = CarAgent(self, start_pos, destination_pos)
            self.car_agents.add(car)
            # Place the car agent on the grid
            self.grid.place_agent(car, start_pos)

    def create_traffic_lights(self):
        """Create traffic light agents at specified positions."""
        traffic_light_positions = [
            (12, 12), (12, 0), (0, 12), (23, 12), (12, 23)
            # Add more traffic lights as needed
        ]
        for pos in traffic_light_positions:
            traffic_light = TrafficLightAgent(self, pos)
            self.traffic_light_agents.add(traffic_light)
            # Place the traffic light agent on the grid
            self.grid.place_agent(traffic_light, pos)

    def is_building(self, pos):
        """Check if a position is occupied by a building."""
        x, y = pos
        return self.buildings_layer[x, y]

    def is_valid_road_direction(self, from_pos, to_pos):
        """Check if movement from from_pos to to_pos follows the road direction."""
        from_x, from_y = from_pos
        to_x, to_y = to_pos
        direction = self.road_direction_layer[from_x, from_y]
        if direction == 'N' and to_y == from_y + 1:
            return True
        if direction == 'S' and to_y == from_y - 1:
            return True
        if direction == 'E' and to_x == from_x + 1:
            return True
        if direction == 'W' and to_x == from_x - 1:
            return True
        return False

    def step(self):
        """Advance the model by one step."""
        # Activate traffic lights
        self.traffic_light_agents.do("step")
        # Activate cars
        self.car_agents.do("step")
        # Increment step counter
        self.steps += 1

In [16]:
def agent_portrayal(agent):
    portrayal = {
        "x": agent.pos[0],
        "y": agent.pos[1],
        "color": "blue",
        "marker": "o",
        "size": 5,
        "alpha": 1.0,
    }
    return portrayal

In [17]:
# Create the SpaceComponent for visualization
SpaceComponent = make_space_component(agent_portrayal)

# Create an instance of the model
model = CityModel(num_cars=5, width=10, height=10)



IndexError: index 10 is out of bounds for axis 0 with size 10

In [8]:
# Create the SolaraViz page for visualization
page = SolaraViz(
    model,
    components=[SpaceComponent],
    name="Simple City Model",
)

# Display the page (in a Jupyter Notebook or Solara app)
page

/Users/pedropc/.pyenv/versions/3.13.0/envs/myenv/lib/python3.13/site-packages/mesa/visualization/mpl_space_drawing.py:272: UserWarning: the following fields are not used in agent portrayal and thus ignored: x, y.
  arguments = collect_agent_data(space, agent_portrayal, size=s_default)
/Users/pedropc/.pyenv/versions/3.13.0/envs/myenv/lib/python3.13/site-packages/mesa/visualization/mpl_space_drawing.py:272: UserWarning: the following fields are not used in agent portrayal and thus ignored: x, y.
  arguments = collect_agent_data(space, agent_portrayal, size=s_default)


Cannot show ipywidgets in text